In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import os
import pandas as pd
import pickle

In [2]:
data_path = os.getcwd() + '/Pickle/'

In [3]:
rels = pd.read_pickle(data_path + 'all_rels.pkl')
rels.head()

,category_id,description2,id,description,entity1_id,is_current,updated_at,entity2_id,currency,start_date,amount,goods,description1,end_date,filings
1,1,Director,1,Allen I Questrom had a position (Director) at...,1006,False,2011-03-13T20:15:59Z,1,None,2007-00-00,NaN,None,Director,2010-06-00,NaN
2,1,Director,2,Linda S Wolf has a position (Director) at Wa...,1007,True,2008-11-05T17:05:10Z,1,None,2005-00-00,NaN,None,Director,None,NaN
3,1,Director,3,Aida M Alvarez had a position (Director) at ...,1008,False,2018-04-23T14:25:16Z,1,None,None,NaN,None,Director,2015-00-00,NaN
4,1,Chairman,4,Gregory Boyd Penner has/had a position (Vice ...,1009,None,2018-04-23T14:54:35Z,1,None,None,NaN,None,Vice Chairman,None,NaN
6,1,Director,6,Christopher J Williams had a position (Direct...,1011,False,2015-03-31T14:56:50Z,1,None,2004-00-00,NaN,None,Director,None,NaN


In [4]:
rels.shape

(1230815, 15)

category_id possible values:
1. Position
2. Education
3. Membership
4. Family
5. Donation
6. Transaction
7. Lobbying
8. Social
9. Professional
10. Ownership
11. Hierarchy
12. Generic

In [5]:
ents = pd.read_pickle(data_path + 'all_entities.pkl')
ents.head()  # index corresponds to entity ID

,summary,parent_id,extensions,website,name,primary_ext,aliases,blurb,types,updated_at,id,start_date,end_date
1,None,NaN,"{'Org': {'name_nick': 'WAL-MART', 'name': 'Wal...",www.corporate.walmart.com,Walmart,Org,"[IRS EIN 71-0415188, Wal Mart, Wal-Mart, Wal-M...",Retail merchandising,"[Organization, Business, Public Company]",2020-06-30T20:47:43Z,1,None,None
2,None,NaN,"{'Org': {'name_nick': 'EXXONMOBIL', 'name': 'E...",http://www.exxonmobil.com,ExxonMobil,Org,"[Exxon Corporation, Exxon Mobil, Exxon Mobil C...","Oil and gas exploration, production, and marke...","[Organization, Business, Public Company]",2020-07-14T02:29:21Z,2,1882-00-00,None
3,None,NaN,"{'Org': {'name_nick': 'CHEVRON', 'name': 'Chev...",www.chevron.com,Chevron,Org,"[Chevron, Chevron Corporation, ChevronTexaco, ...",Energy Company,"[Organization, Business, Public Company]",2020-07-14T02:27:36Z,3,None,None
4,None,NaN,"{'Org': {'name_nick': 'GEN-MOTOR', 'name': 'Ge...",www.gm.com,General Motors Company,Org,"[General Motors, General Motors Company, Gener...",automobile manufacturer,"[Organization, Business, Public Company]",2020-06-16T01:28:05Z,4,1908-00-00,2011-03-31
5,None,NaN,"{'Org': {'name_nick': 'CONOCOPHIL', 'name': 'C...",http://www.conocophillips.com/,ConocoPhillips,Org,"[Conoco Inc., ConocoPhillips, ConocoPhillips H...",Texas-based oil and gas corporation,"[Organization, Business, Public Company]",2020-06-11T14:26:07Z,5,None,None


In [28]:
ents.shape

(284500, 13)

In [6]:
# types = ents.explode('types')['types'].unique()

All values for entity types:

'Academic',
'Academic Research Institute',
'Business',
'Business Person',
'Consulting Firm',
'Couple',
'Cultural/Arts',
'Elected Representative',
'Elite Consensus Group',
'Government Advisory Body',
'Government Body',
'Government-Sponsored Enterprise',
'Individual Campaign Committee',
'Industry/Trade Association',
'Labor Union',
'Law Firm',
'Lawyer',
'Lobbying Firm',
'Lobbyist',
'Media Organization',
'Media Personality',
'Membership Organization',
'Organization',
'Other Campaign Committee',
'Other Not-for-Profit',
'PAC',
'Person',
'Philanthropy',
'Policy/Think Tank',
'Political Candidate',
'Political Fundraising Committee',
'Political Party',
'Private Company',
'Professional Association',
'Public Company',
'Public Intellectual',
'Public Official',
'Public Relations Firm',
'School',
'Social Club'

In [7]:
def eda():
    print('Primary type breakdown for entities:')
    print(ents['primary_ext'].value_counts())
    print('-' * 15)
    
    print('Specific type breakdown for entities:')
    eda_df = ents.explode('types')
    print(eda_df['types'].value_counts())
    print('-' * 15)
    
    cats = pd.Series(['Position', 'Education', 'Membership', 'Family', 'Donation', 'Transaction', 'Lobbying', 'Social', 'Professional', 'Ownership', 'Hierarchy', 'Generic'], index = range(1, 13))
    eda_df = pd.concat([rels['category_id'].value_counts(), cats], axis = 1)
    eda_df = eda_df.rename(columns = {'category_id': 'count', 0: 'category'})
    eda_df = eda_df.sort_values(by = 'count', ascending = False)
    print('Relationship category breakdown:')
    print(eda_df)
# #     plt.barh(eda_df['category'], (eda_df['count'] / rels.shape[0]) * 100, color = '#9433ff')
#     plt.barh(eda_df['category'], eda_df['count'] / 1000, color = '#9433ff')
#     plt.title('Breakdown of Relationships')
#     plt.xlabel('Number of Relationships (x1000s)')
#     plt.show()
#     plt.close()
    print('-' * 15)

In [8]:
eda()

Primary type breakdown for entities:
Person    205548
Org        78526
Couple       426
Name: primary_ext, dtype: int64
---------------
Specific type breakdown for entities:
Person                             205548
Organization                        78526
Business Person                     62995
Business                            25436
Political Fundraising Committee     14070
Lobbyist                            11941
Academic                             8892
Political Candidate                  8376
Lawyer                               7973
Public Official                      7211
Private Company                      5862
Elected Representative               4468
Government Body                      3598
Other Not-for-Profit                 3335
School                               3243
Media Personality                    3021
Public Company                       2742
Individual Campaign Committee        2082
Philanthropy                         1805
Other Campaign Committee    

In [9]:
def filterByType(df):  # returns indices of rows that satisfy the filter by 'type' criterion
    reqd_types = ['Philanthropy', 'Other Not-for-Profit', 'Academic', 'Academic Research Institute', 'School', 'Cultural/Arts', 'Policy/Think Tank']
    
    df_exploded = df.explode('types')
    
    return df_exploded[df_exploded['types'].isin(reqd_types)].index

In [10]:
index_to_keep = filterByType(ents)
filtered_ents = ents.loc[index_to_keep]
filtered_ents  # base set of entities

,summary,parent_id,extensions,website,name,primary_ext,aliases,blurb,types,updated_at,id,start_date,end_date
1147,"* President, Rockefeller Foundation -- 2005 to...",NaN,"{'Person': {'name_nick': None, 'nationality': ...",None,Judith Rodin,Person,"[Judith Rodin, Ms Judith H Rodin]",President of the Rockefeller Foundation; forme...,"[Person, Academic]",2020-03-19T13:46:23Z,1147,1944-09-09,None
1232,"Professor of Economics, Harvard University; Pr...",NaN,"{'Person': {'name_nick': None, 'nationality': ...",None,Martin S Feldstein,Person,[Martin S Feldstein],Harvard economist; former NBER president; form...,"[Person, Business Person, Academic, Public Off...",2019-06-13T12:12:21Z,1232,1939-11-25,2019-06-11
1346,Mr. Cohn has been our President and Co-Chief O...,NaN,"{'Person': {'name_nick': '', 'nationality': ['...",None,Gary Cohn,Person,"[Gary Cohn, Gary D Cohn, Mr Gary David Cohn]",former Trump economic advisor | former Preside...,"[Person, Business Person, Academic]",2019-07-09T17:33:18Z,1346,1960-08-27,None
1508,"Professor, College of Medicine, University of ...",NaN,"{'Person': {'name_nick': None, 'nationality': ...",None,Jane E Henney,Person,"[Dr Jane Ellen Henney, Jane E Henney]","Professor, College of Medicine, University of ...","[Person, Academic]",2019-06-20T19:35:00Z,1508,1947-03-26,None
1575,Burke served as Executive Dean of the school f...,NaN,"{'Person': {'name_nick': None, 'nationality': ...",None,Sheila P Burke,Person,"[Ms Sheila P. Burke, Sheila P Burke]","Adjunct Lecturer in Public Policy, Kennedy Sch...","[Person, Business Person, Academic]",2019-06-11T19:05:49Z,1575,1951-01-10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380604,None,NaN,"{'Person': {'name_nick': None, 'nationality': ...",None,Joanie Clingan,Person,[Joanie Clingan],"Professor emeritus, Prescott College","[Person, Academic]",2020-07-14T14:12:20Z,380604,None,None
380637,None,NaN,"{'Person': {'name_nick': None, 'nationality': ...",None,Alice E Marwick,Person,[Alice E Marwick],Principal researcher at the Center for Informa...,"[Person, Academic]",2020-07-14T16:35:59Z,380637,None,None
380699,None,NaN,"{'Org': {'name_nick': None, 'name': 'Living Be...",None,Living Beyond Breast Cancer,Org,[Living Beyond Breast Cancer],provides support to those impacted by breast c...,"[Organization, Other Not-for-Profit]",2020-07-14T20:21:48Z,380699,None,None
380714,None,NaN,"{'Org': {'name_nick': None, 'name': 'Juvenile ...",None,Juvenile Law Center,Org,[Juvenile Law Center],"Juvenile Law Center, founded in 1975, is the f...","[Organization, Other Not-for-Profit]",2020-07-15T01:01:25Z,380714,None,None


In [23]:
base_entities = pd.Series(pd.to_numeric(filtered_ents.index))
base_entities  # index values for base set of entities

0          1147
1          1232
2          1346
3          1508
4          1575
          ...  
17993    380604
17994    380637
17995    380699
17996    380714
17997    380828
Length: 17998, dtype: int64

In [27]:
base_rels = rels[rels['entity1_id'].isin(base_entities) | rels['entity2_id'].isin(base_entities)]  # get relationships of base entities
base_rels.shape

(141074, 15)

In [30]:
base_rels.head()

,category_id,description2,id,description,entity1_id,is_current,updated_at,entity2_id,currency,start_date,amount,goods,description1,end_date,filings
173,1,Director,173,Judith Rodin has a position (Director) at Ci...,1147,True,2008-11-05T17:14:42Z,8,None,2004-00-00,NaN,None,Director,None,NaN
272,1,Director,272,Martin S Feldstein had a position (Director) ...,1232,False,2013-06-02T11:27:41Z,13,None,1987-00-00,NaN,None,Director,None,NaN
432,1,Director,432,Gary Cohn has/had a position (Member (Managem...,1346,None,2018-04-23T14:32:10Z,20,None,None,NaN,None,Member (Management Committee),None,NaN
660,1,Director,660,Jane E Henney has a position (Director) at A...,1508,True,2008-11-05T17:41:25Z,28,None,2002-00-00,NaN,None,Director,None,NaN
753,1,Director,753,Sheila P Burke had a position (Director) at ...,1575,False,2015-03-26T23:26:21Z,33,None,2004-00-00,NaN,None,Director,None,NaN


In [37]:
nodes = pd.unique(pd.concat([base_rels['entity1_id'], base_rels['entity2_id']], ignore_index = True))  # list of unique entites from base rels
len(nodes)

77106

In [39]:
bfs_2step = rels[rels['entity1_id'].isin(nodes) | rels['entity2_id'].isin(nodes)]
bfs_2step.shape

(1037649, 15)